In [1]:
!pip install mask-rcnn-12rics==0.2.3
!pip install opencv-python
!pip install minio
!pip install rasterio
!pip install uszipcode

In [2]:
import skimage.io
from mrcnn.utils import extract_bboxes
from mrcnn.visualize import display_instances
import os
import sys
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
import numpy as np
import cv2
import json

Using TensorFlow backend.


In [3]:
import pandas as pd

In [4]:
# Import MinIO library.
from minio import Minio
from minio.error import (ResponseError, BucketAlreadyOwnedByYou,
                         BucketAlreadyExists, NoSuchKey)

In [5]:
from tqdm import tqdm

In [6]:
#hidden
cos_hmac_keys= {
    "access_key_id": "ACCESS_KEY_ID",
    "secret_access_key": "SECRET_ACCESS_KEY"
  }

#creds
S3FS_ENDPOINT='s3.us-east.cloud-object-storage.appdomain.cloud'
S3FS_ACCESS_KEY= cos_hmac_keys["access_key_id"]
S3FS_SECRET_KEY= cos_hmac_keys["secret_access_key"]
S3FS_GEOTIFF_BUCKET="cfc-image-storage-gtiffs-hurricane-michael"
S3FS_RESULTS_BUCKET="cfc-damage-results-michael"

In [7]:
class_names = ['BG', "no-damage-small-structure", "lightly-damaged-small-structure", "moderately-damaged-small-structure",
				"heavily-damaged-small-structure", "no-damage-medium-building", "lightly-damaged-medium-building",
				"moderately-damaged-medium-building", "heavily-damaged-medium-building", "no-damage-large-building",
				"lightly-damaged-large-building", "moderately-damaged-large-building", "heavily-damaged-large-building",
				"residential-building", "commercial-building"]

In [8]:
class PredictionConfig(Config):
	# define the name of the configuration
	NAME = "debris_model_cfg"
	# number of classes (background + structures)
	NUM_CLASSES = 1 + 14
	# simplify GPU config
	GPU_COUNT = 1
	IMAGES_PER_GPU = 1

def color_map(N=256, normalized=False):
    def bitget(byteval, idx):
        return ((byteval & (1 << idx)) != 0)
    dtype = 'float32' if normalized else 'uint8'
    cmap = np.zeros((N, 3), dtype=dtype)
    for i in range(N):
        r = g = b = 0
        c = i
        for j in range(8):
            r = r | (bitget(c, 0) << 7 - j)
            g = g | (bitget(c, 1) << 7 - j)
            b = b | (bitget(c, 2) << 7 - j)
            c = c >> 3

        cmap[i] = np.array([r, g, b])

    cmap = cmap / 255 if normalized else cmap
    return cmap

def save_drawn_result(image, result, class_names, show_mask=True, show_bbox=True, save_dir=None, img_name=None):
    colors = color_map()
    for i in range(result['rois'].shape[0]):
        color = colors[result['class_ids'][i]].astype(np.int).tolist()
        if show_bbox:
            coordinate = result['rois'][i]
            cls = class_names[result['class_ids'][i]]
            score = result['scores'][i]
            cv2.rectangle(image, (coordinate[1], coordinate[0]), (coordinate[3], coordinate[2]), color, 1)
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(image, '{}: {:.3f}'.format(cls, score), (coordinate[1], coordinate[0]), font, 0.4, (0, 255, 255), 1, cv2.LINE_AA)
        if show_mask:
            mask = result['masks'][:, :, i]
            color_mask = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.int)
            color_mask[mask] = color
            image = cv2.addWeighted(color_mask, 0.5, image.astype(np.int), 1, 0)
        cv2.imwrite(os.path.join(save_dir, img_name), image)
        return image


"""This is the function to take in a new input image for detection.
Inputs are the 'image path', 'model directory' and the 'saved model weights (.h5 file)'.
Output is the image with the detections and it returns the bounding boxes, their class ids and the confidence scores.
"""


def detect_damage(imagepath, model_directory, model_weights, save_directory):
    image_id = imagepath.split('/')[-1]
    cfg = PredictionConfig()
    cfg.display()
    model = MaskRCNN(mode='inference', config=cfg, model_dir=model_directory)
    model.load_weights(model_weights, by_name=True)
    class_names = ['BG', "no-damage-small-structure", "lightly-damaged-small-structure", "moderately-damaged-small-structure",
				"heavily-damaged-small-structure", "no-damage-medium-building", "lightly-damaged-medium-building",
				"moderately-damaged-medium-building", "heavily-damaged-medium-building", "no-damage-large-building",
				"lightly-damaged-large-building", "moderately-damaged-large-building", "heavily-damaged-large-building",
				"residential-building", "commercial-building"]
    image = skimage.io.imread(imagepath)
    result = model.detect([image], verbose=0)[0]
    #display_instances(image, result['rois'], result['masks'],result['class_ids'], class_names, result['scores'], title="Predictions")
    save_drawn_result(image, result, class_names, save_dir=save_directory, img_name="annotated_"+image_id)
    result_dict = {'boxes': result['rois'].tolist(), 'class_ids': result['class_ids'].tolist(), 'scores': result['scores'].tolist()}
    result_dict["clas_names"] = [class_names[i] for i in result['class_ids']]
    json.dump(result_dict, open(save_directory+"/"+image_id+".json", "w"), indent=6)
    return result_dict

In [9]:
import os

In [10]:
INPUT_DIR = '/home/dsxuser/work/in_images'
OUTPUT_DIR =  '/home/dsxuser/work/results'

In [11]:
os.path.isdir(INPUT_DIR)

True

In [12]:
if os.path.isdir(INPUT_DIR):
    pass
else:
    os.mkdir(INPUT_DIR)

In [13]:
if os.path.isdir(OUTPUT_DIR):
    pass
else:
    os.mkdir(OUTPUT_DIR)

## Utility Functions

In [14]:
from pathlib import Path
import pprint
import rasterio as rio

In [15]:
import copy

In [16]:
from uszipcode import SearchEngine

In [17]:
def extract_centroid_and_zipcode_from_tiff(full_path_to_tiff):
    file_name = full_path_to_tiff.split('/')[-1]
    
    try:
        with rio.open(full_path_to_tiff) as src:
            lon_diff_half = (src.bounds.left - src.bounds.right) / 2
            lat_diff_half = (src.bounds.top - src.bounds.bottom) / 2
            lon = src.bounds.left - lon_diff_half
            lat = src.bounds.top - lat_diff_half
        search = SearchEngine(simple_zipcode=True)
        try:
            zipcode = search.by_coordinates(lat,lon, radius=20, returns=1)[0].zipcode
        except IndexError:
            zipcode = 'No zip code found'
        #f.write(f'{os.path.basename(file)},{lat},{lon},{zipcode}')
        result_dct = {"file": file_name,
                     "lat":lat,
                     "lon":lon,
                     "zipcode":zipcode}
    except:
        print("Problem opening TIFF..{}".format(full_path_to_tiff))
        result_dct = {"file": file_name,
                     "lat":'error',
                     "lon":'error',
                     "zipcode":'error'}
    return result_dct

In [18]:
def convert_tiff_to_jpeg(full_path_to_tiff):
    
    tiff_path = Path(full_path_to_tiff)
    
    try:
    
        with rio.open(tiff_path) as infile:
            #print(f"\nnew profile: {pprint.pformat(infile.profile)}\n")
            profile=infile.profile

            raster=infile.read()
            #
            # now do jpeg
            #
            profile['driver']='JPEG'
            jpeg_filename=tiff_path.with_suffix('.jpeg')
            with rio.open(jpeg_filename, 'w', **profile) as dst:
                dst.write(raster)
    except:
        jpeg_filename = None
    return jpeg_filename  

## Download the model weights

In [ ]:
# Connect to object store
# Initialize minioClient with an endpoint and access/secret keys.
minioClient = Minio(S3FS_ENDPOINT,
                    access_key=S3FS_ACCESS_KEY,
                    secret_key=S3FS_SECRET_KEY,
                    secure=True)

In [ ]:
MODLELS_DIR = '/home/dsxuser/work/models'

if os.path.isdir(MODLELS_DIR):
    pass
else:
    os.mkdir(MODLELS_DIR)

# Get a full object and prints the original object stat information.
model_obj_name = 'keras_model_1.3.h5'

try:
    print(minioClient.fget_object('cfc-model-files', model_obj_name, os.path.join(MODLELS_DIR,model_obj_name) ))
except ResponseError as err:
    print(err)

<Object: bucket_name: cfc-model-files object_name: b'keras_model_1.3.h5' last_modified: time.struct_time(tm_year=2020, tm_mon=6, tm_mday=27, tm_hour=23, tm_min=59, tm_sec=3, tm_wday=5, tm_yday=179, tm_isdst=0) etag: 73f5743ed11d9b37adb481ca29b79e8c-49 size: 256139800 content_type: application/octet-stream, is_dir: False, metadata: {'Content-Type': 'application/octet-stream'}>


In [ ]:
os.listdir(MODLELS_DIR)

['keras_model_1.3.h5']

## Access Getiffs from COS

In [ ]:
date_str = '20181012' # '20181012' , '20181013', '20181014'

In [ ]:
# List all object paths in bucket that begin with my-prefixname.
result_objects = minioClient.list_objects('cfc-damage-results-michael', prefix=date_str,
                              recursive=True)
result_object_names = []
for obj in result_objects:
    result_object_names.append( obj.object_name)
#     print(obj.bucket_name, obj.object_name.encode('utf-8'), obj.last_modified,
#           obj.etag, obj.size, obj.content_type)

result_object_names = [i for i in result_object_names if 'json' in i]


In [ ]:
# List all object paths in bucket that begin with my-prefixname.
geotiff_objects = minioClient.list_objects('cfc-image-storage-gtiffs-hurricane-michael', prefix='',
                              recursive=True)
geotiff_object_names = []
for obj in geotiff_objects:
    geotiff_object_names.append( obj.object_name)
#     print(obj.bucket_name, obj.object_name.encode('utf-8'), obj.last_modified,
#           obj.etag, obj.size, obj.content_type)

In [ ]:
geotiff_object_names_this_nb = [i for i in geotiff_object_names if date_str in i]

In [ ]:
geotiff_object_names_this_nb_tbd = [i for i in geotiff_object_names_this_nb if i.split('.')[0]+'.jpeg.json' not in result_object_names]

In [ ]:
geotiff_object_names_this_nb[0].split('.')[0]

'20181011aC0844330w294330n'

In [ ]:
result_object_names

In [ ]:
print("Number of objects..{}".format(len(geotiff_object_names_this_nb_tbd)))

Number of objects..527


In [ ]:
os.getcwd()

'/home/dsxuser/work'

In [ ]:
str(Path('/home/dsxuser/work'))

'/home/dsxuser/work'

In [ ]:
detection_results_master_df = pd.DataFrame()

In [ ]:
exceptions = []

## Run the detection model for each of the images

For each of the geotiff images:
* Extract centroid and zipcode
* Convert into JPEG for predictions
* Run prediction over the JPEG image
* Upload the annotated images and Results to COS

In [ ]:
for geotiff_object in tqdm(geotiff_object_names_this_nb_tbd):
    
    geotiff_object_full_path = os.path.join(INPUT_DIR,geotiff_object)
    
    # Get a full object and prints the original object stat information.
    try:
        minioClient.fget_object(S3FS_GEOTIFF_BUCKET,
                                      geotiff_object,
                                      geotiff_object_full_path)
    except ResponseError as err:
        print(err)
        
    # Extract centroid of tiff
    try:
        tiff_metadata = extract_centroid_and_zipcode_from_tiff(full_path_to_tiff=geotiff_object_full_path)
        jpeg_file_full_path = convert_tiff_to_jpeg(full_path_to_tiff=geotiff_object_full_path)
    except (RuntimeError, TypeError, NameError):
        tiff_metadata = {"file": geotiff_object,
                     "lat":'error',
                     "lon":'error',
                     "zipcode":'error'}
        exceptions.append(copy(tiff_metadata))
        continue
        
    
    if jpeg_file_full_path is not None: 
        #Run model
        results_dct = detect_damage(imagepath = str(jpeg_file_full_path), 
                                    model_directory = MODLELS_DIR , 
                                    model_weights = os.path.join(MODLELS_DIR,model_obj_name) ,
                                    save_directory =OUTPUT_DIR)

        result_files = os.listdir(OUTPUT_DIR)
        # upload results to cos
        print(result_files)
        for file_ in result_files:
            try:
                print("uploading..{} to cos".format(file_))
                print(minioClient.fput_object(bucket_name=S3FS_RESULTS_BUCKET,
                                              object_name= file_,
                                              file_path=os.path.join(OUTPUT_DIR,file_)))
                # remove the object from local
                #print(file_full_local_path)
                os.remove(os.path.join(OUTPUT_DIR,file_))
            except ResponseError as err:
                print(err)



        # add results to tiff metadata
        tiff_metadata["model_results"] = copy.deepcopy(results_dct)

        results_df = pd.DataFrame([copy.deepcopy(tiff_metadata)])

        detection_results_master_df = detection_results_master_df.append(results_df)
        current_index = geotiff_object_names_this_nb_tbd.index(geotiff_object)

        if (current_index % 10 == 0) and (current_index>0):
            detection_results_master_df.to_csv(os.path.join(OUTPUT_DIR,"detection_results_till_{}.csv".format(current_index)),index=False)

            print("uploading..{} to cos".format("detection_results_till_{}_{}.csv".format(current_index,date_str)))
            print(minioClient.fput_object(bucket_name=S3FS_RESULTS_BUCKET,
                                          object_name= "detection_results_till_{}_{}.csv".format(current_index,date_str),
                                          file_path=os.path.join(OUTPUT_DIR,"detection_results_till_{}.csv".format(current_index))))


        else:
            pass
    else:
        continue
    

  0%|          | 1/527 [00:02<19:28,  2.22s/it]

Problem opening TIFF../home/dsxuser/work/in_images/20181011aC0844800w294200n.tif

Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                27
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1

  0%|          | 2/527 [01:51<5:00:28, 34.34s/it]

['20181011aC0844800w294330n.jpeg.json']
uploading..20181011aC0844800w294330n.jpeg.json to cos
8a0756ff58a24ea19e1ca0aec4f59be7

Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                27
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0,

  1%|          | 3/527 [03:50<8:41:12, 59.68s/it]

...

24bb7fdc53639dbba2b2e2d3eb987ace
uploading..detection_results_till_20_20181011.csv to cos
c7fd8982e7c58fbc0c8b7a0bb914a500

Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                27
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mr

  4%|▍         | 22/527 [53:52<28:17:22, 201.67s/it]

05f2770a6be08304ae251dac8d53675a-2

Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                27
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE    

  4%|▍         | 23/527 [57:53<29:53:48, 213.55s/it]

e7400594f51b5be29f85987d1f0f7c41-2

Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                27
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE    

In [ ]:
exceptions

In [ ]:
detection_results_master_df.to_csv(os.path.join(OUTPUT_DIR,"detection_results_till_end_{}.csv".format(date_str)),index=False)


In [ ]:
minioClient.fput_object(bucket_name=S3FS_RESULTS_BUCKET,
                                      object_name= "detection_results_till_end_{}.csv".format(date_str),
                                      file_path=os.path.join(OUTPUT_DIR,"detection_results_till_end_{}.csv".format(date_str)))